In [1]:
import torch
from datasets import load_dataset
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())
    print(torch.cuda.current_device())
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, RobertaTokenizerFast, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
import evaluate
path = '/app/Data/'
#model_path = 'google-bert/bert-base-uncased'
# tokenizer = BertTokenizerFast.from_pretrained(model_path)
model_path = 'FacebookAI/roberta-base'
tokenizer = RobertaTokenizerFast.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
NVIDIA GeForce RTX 3080
0


2024-04-13 15:55:30.732024: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 15:55:31.423195: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 15:55:32.770884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
arrow_file_name = "data-00000-of-00001"
def load_data(path,tokenized=False):
    if tokenized:
        full_ds = load_dataset('arrow',data_files={
            'train_r':path+f'/tokenized/train_ds_r/{arrow_file_name}.arrow',
            'train_e':path+f'/tokenized/train_ds_e/{arrow_file_name}.arrow',
            'test':path+f'/tokenized/test_ds/{arrow_file_name}.arrow',
            'val':path+f'/tokenized/val_ds/{arrow_file_name}.arrow'
        })
    else:
        full_ds = load_dataset('arrow',data_files={
            'train_r':path+f'/train_ds_r/{arrow_file_name}.arrow',
            'train_e':path+f'/train_ds_e/{arrow_file_name}.arrow',
            'test':path+f'/test_ds/{arrow_file_name}.arrow',
            'validation':path+f'/val_ds/{arrow_file_name}.arrow'
        })
    return full_ds


### Init Evaluation metrics, Data Collator and Model

In [3]:
tokenized_ds = load_data(path,tokenized=True)

In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {
    0:"ham",
    1:"spam",
}

label2id = {
    "ham":0,
    "spam":1,
}

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)

eval_class_weights = torch.tensor([0.5, 1.0])

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
class CustomTrainer(Trainer):
    def __init__(self, *args, eval_class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # eval_class_weights should be a tensor of shape [num_labels].
        self.eval_class_weights = eval_class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        # Call the original compute_loss
        outputs = model(**inputs)
        logits = outputs.get('logits')
        labels = inputs.get('labels')
        if self.eval_class_weights is not None and self.args.do_eval:
            # Only modify loss computation during evaluation
            eval_class_weights_device = self.eval_class_weights.to(model.device)
            loss_fct = torch.nn.CrossEntropyLoss(weight=eval_class_weights_device)
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        else:
            # Default behavior
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

## Model Training

### Random Undersampling

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)

to_torch_compile = True # Change to True only if on linux
training_args = TrainingArguments(
    output_dir="/app/model_checkpoints",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    optim="adamw_bnb_8bit", # Supposedly better than AdamW while using less space
    gradient_accumulation_steps=2, # Increases Effective Batch Size for smoother gradient descent
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.1,
    evaluation_strategy="steps",
    push_to_hub=False,
    metric_for_best_model="eval_loss",
    logging_steps=100,
    save_steps=100,
    eval_steps=100,
    save_total_limit=5,
    load_best_model_at_end=True,
    torch_compile = to_torch_compile
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train_r"],
    eval_dataset=tokenized_ds["val"],
    eval_class_weights=eval_class_weights,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
100,0.278200,0.182699,0.926129
200,0.171000,0.141325,0.960073
300,0.139000,0.207691,0.952747
400,0.109400,0.110401,0.974847
500,0.130100,0.154370,0.963614
600,0.117500,0.097782,0.975214
700,0.096500,0.117731,0.974969
800,0.078600,0.132617,0.975092
900,0.088300,0.084679,0.980952
1000,0.106000,0.129337,0.978022


TrainOutput(global_step=6928, training_loss=0.03994914622034089, metrics={'train_runtime': 5352.8309, 'train_samples_per_second': 20.708, 'train_steps_per_second': 1.294, 'total_flos': 2.68121676520512e+16, 'train_loss': 0.03994914622034089, 'epoch': 4.0})

In [15]:
trainer.save_model("/app/models/test_r")

### Embedding Undersampling

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)

to_torch_compile = True # Change to True only if on linux
training_args = TrainingArguments(
    output_dir="/app/model_checkpoints",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    optim="adamw_bnb_8bit", # Supposedly better than AdamW while using less space
    gradient_accumulation_steps=2, # Increases Effective Batch Size for smoother gradient descent
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.1,
    evaluation_strategy="steps",
    push_to_hub=False,
    metric_for_best_model="eval_loss",
    logging_steps=100,
    save_steps=100,
    eval_steps=100,
    save_total_limit=5,
    load_best_model_at_end=True,
    torch_compile = to_torch_compile
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train_e"],
    eval_dataset=tokenized_ds["val"],
    eval_class_weights=eval_class_weights,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [7]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
100,0.325600,0.256904,0.913065
200,0.199200,0.185128,0.956410
300,0.161200,0.178501,0.956532
400,0.128700,0.134404,0.964713
500,0.124100,0.142743,0.965079
600,0.106600,0.138370,0.967033
700,0.140800,0.169818,0.943101
800,0.143600,0.100224,0.968498
900,0.103900,0.131701,0.974115
1000,0.102800,0.101501,0.976557


In [18]:
trainer.save_model("/app/models/test_e")

## Model Evaluation

### Embedding Undersampling

In [3]:
tokenized_ds = load_data(path,tokenized=True)
tokenized_test_ds = tokenized_ds['test']
classifier = pipeline('text-classification',model='/app/models/test_e', device=torch.cuda.current_device())
def decode_tokens(example):
    tokens = example['text']
    # label_map = {0:"ham",1:"spam"}
    text = tokenizer.decode(tokens,skip_special_tokens=True)
    # label = label_map[example['label']]
    return {'text':text}

tokenized_test_ds = tokenized_test_ds.map(decode_tokens)

In [6]:
from evaluate import evaluator
task_evaluator = evaluator('text-classification')
results = task_evaluator.compute(
    model_or_pipeline=classifier,
    data=tokenized_test_ds,
    metric=evaluate.combine(['accuracy','recall','precision','f1']),
    label_mapping=label2id,
)

In [7]:
results

{'accuracy': 0.9881040892193309,
 'recall': 0.9762845849802372,
 'precision': 0.9762845849802372,
 'f1': 0.9762845849802372,
 'total_time_in_seconds': 46.33630791600001,
 'samples_per_second': 87.08074038429608,
 'latency_in_seconds': 0.011483595518215615}

### Random Undersampling

In [16]:
classifier = pipeline('text-classification',model='/app/models/test_r', device=torch.cuda.current_device())
from evaluate import evaluator
task_evaluator = evaluator('text-classification')
results = task_evaluator.compute(
    model_or_pipeline=classifier,
    data=tokenized_test_ds,
    metric=evaluate.combine(['accuracy','recall','precision','f1']),
    label_mapping=label2id,
)
print(results)

{'accuracy': 0.9866171003717472, 'recall': 0.9901185770750988, 'precision': 0.9579349904397706, 'f1': 0.9737609329446064, 'total_time_in_seconds': 39.899076257999695, 'samples_per_second': 101.13016085656844, 'latency_in_seconds': 0.009888246904089144}


In [17]:
results

{'accuracy': 0.9866171003717472,
 'recall': 0.9901185770750988,
 'precision': 0.9579349904397706,
 'f1': 0.9737609329446064,
 'total_time_in_seconds': 39.899076257999695,
 'samples_per_second': 101.13016085656844,
 'latency_in_seconds': 0.009888246904089144}